# 基本设置

In [30]:
import sys
import re
import time
import string

import xlrd

%matplotlib inline
import numpy as np
import pandas as pd
import pre_cor_1
import pre_cor
import os
from sqlalchemy import create_engine
from pandas.io import sql
import re 
import warnings
warnings.filterwarnings('ignore')

# 读取文件

In [2]:
#这两个文件是自行爬取的正文
# file_list1 = ['20180723_revise_clearn.xlsx','20180725_clearn.xlsx']

In [3]:
# cir_data_raw_1 = pd.DataFrame()
# for file_name in file_list1:
#     tmp_data = pd.read_excel('data/result/{0}'.format(file_name))
#     print(tmp_data.shape, file_name)
#     cir_data_raw_1 = pd.concat([cir_data_raw_1, tmp_data], axis = 0)

# print('总数：', cir_data_raw_1.shape)
# cir_data_raw_1 = cir_data_raw_1[cir_data_raw_1['label'] != '噪音']
# cir_data_raw_1 = cir_data_raw_1[cir_data_raw_1['label'] != '产品销售']
# cir_data_raw_1 = cir_data_raw_1[cir_data_raw_1['label'] != '行业']
# print('去除噪音后，总数：', cir_data_raw_1.shape)

# # print(cir_data_raw_1[cir_data_raw_1['title']=='听说出来了一款保证续保的医疗险?真相如何?'])
# cir_data_raw_1.head()

In [4]:
# cir_data_raw_1['label'].value_counts()

In [5]:
# #这些文件是自身带有正文的文件
# file_list2 = ['20180727_circ_class_1.xlsx', '20180803_circ_class_1.xlsx', 
#               '20180803_circ_class_2.xlsx', '20180803_circ_class_3.xlsx', 
#               '20180807_circ_class_1.xlsx'] # 多个 sheet

In [6]:
# cir_data_raw_2 = pd.DataFrame()
# for file_name in file_list2:
#     currentPath  = 'data/raw_data/{0}'.format(file_name)
#     excel = xlrd.open_workbook(currentPath)
#     sheet_names = [sheet.name for sheet in excel.sheets()]
#     for sheet_name in sheet_names:    
#         tmp_data = pd.read_excel(currentPath, sheet_name)
#         if 'label' in tmp_data.columns:
#             tmp_data = tmp_data[['label', 'title', 'content']]
#             print('去空值前： ', tmp_data.shape, file_name, sheet_name)
#             tmp_data = tmp_data.dropna(subset = ['title'], axis = 0)
#             print('去空值后： ', tmp_data.shape, file_name, sheet_name)
#             cir_data_raw_2 = pd.concat([cir_data_raw_2, tmp_data], axis = 0)

# cir_data_raw_2['label'] = cir_data_raw_2['label'].apply(lambda x:x.strip())    
# print('总数：', cir_data_raw_2.shape)
# cir_data_raw_2.head()

In [7]:
# cir_data_raw_2['label'].value_counts()

## 预测正确数据

In [8]:
file_list_1 = ['class_predict_result_20180806_c.xlsx',]

In [9]:
cir_data_raw_1 = pd.DataFrame()
for file_name in file_list_1:
    currentPath  = 'data/20180808/{0}'.format(file_name)
    excel = xlrd.open_workbook(currentPath)
    sheet_names = [sheet.name for sheet in excel.sheets()]
    for sheet_name in sheet_names:    
        tmp_data = pd.read_excel(currentPath, sheet_name)
        tmp_data = tmp_data[tmp_data['R_W'] == 'Right']
        print('去空值前： ', tmp_data.shape, file_name, sheet_name)
        tmp_data = tmp_data[tmp_data['备注'] != '删除']
#         tmp_data = tmp_data.dropna(subset = ['raw_title'], axis = 0)
        print('去空值后： ', tmp_data.shape, file_name, sheet_name)
        cir_data_raw_1 = pd.concat([cir_data_raw_1, tmp_data], axis = 0)

cir_data_raw_1['label'] = cir_data_raw_1['label'].apply(lambda x:x.strip())    
cir_data_raw_1 = cir_data_raw_1[['label', 'raw_title', 'raw_content']]
cir_data_raw_1.columns = ['label', 'title', 'content']
print('总数：', cir_data_raw_1.shape)
cir_data_raw_1.head()

去空值前：  (715, 7) class_predict_result_20180806_c.xlsx 公司内部管理
去空值后：  (705, 7) class_predict_result_20180806_c.xlsx 公司内部管理
去空值前：  (3073, 7) class_predict_result_20180806_c.xlsx 噪音
去空值后：  (3046, 7) class_predict_result_20180806_c.xlsx 噪音
去空值前：  (480, 7) class_predict_result_20180806_c.xlsx 行业
去空值后：  (478, 7) class_predict_result_20180806_c.xlsx 行业
去空值前：  (1463, 7) class_predict_result_20180806_c.xlsx 消费服务
去空值后：  (1462, 7) class_predict_result_20180806_c.xlsx 消费服务
去空值前：  (434, 7) class_predict_result_20180806_c.xlsx 资本市场
去空值后：  (430, 7) class_predict_result_20180806_c.xlsx 资本市场
去空值前：  (1307, 7) class_predict_result_20180806_c.xlsx 监管
去空值后：  (1287, 7) class_predict_result_20180806_c.xlsx 监管
去空值前：  (201, 7) class_predict_result_20180806_c.xlsx 产品销售
去空值后：  (176, 7) class_predict_result_20180806_c.xlsx 产品销售
去空值前：  (1323, 7) class_predict_result_20180806_c.xlsx 其他相关报道
去空值后：  (1312, 7) class_predict_result_20180806_c.xlsx 其他相关报道
总数： (8896, 3)


,label,title,content
2,公司内部管理,中华保险因未按规定报送可疑交易报告等遭央行罚9万,中华保险因未按规定报送可疑交易报告等遭央行罚9万\n\r2018.01.04 09:17:...
4,公司内部管理,中国人寿北京违法欺骗投保人_保险_金融频道首页_财经网,中国经济网北京1月2日讯北京保监局于2017年12月27日对中国人寿保险股份有限公司北京市...
5,公司内部管理,中国人寿北京市分公司因欺骗投保人被保监局罚款30万元,中国人寿\n\r北京市分公司因欺骗投保人被保监局罚款30万元\n\r千龙网北京1月2日讯 ...
6,公司内部管理,中国人寿|北京保监局_新浪财经_新浪网,中国经济网北京1月2日讯 北京保监局于2017年12月27日对\n\r中国人寿\n\r...
8,公司内部管理,中国人保分公司涉嫌编制虚假报告 被保监会罚款27万,中国人保分公司涉嫌编制虚假报告 被保监会罚款27万\n\r1\n评论\n\r2018-01...


In [10]:
cir_data_raw_1['label'].value_counts()

噪音        3046
消费服务      1462
其他相关报道    1312
监管        1287
公司内部管理     705
行业         478
资本市场       430
产品销售       176
Name: label, dtype: int64

## 预测错误修正后数据

In [11]:
file_list_2 = ['20180807_class_predict_result_check.xlsx',]

In [12]:
cir_data_raw_2 = pd.DataFrame()
for file_name in file_list_2:
    currentPath  = 'data/20180808/{0}'.format(file_name)
    excel = xlrd.open_workbook(currentPath)
    sheet_names = [sheet.name for sheet in excel.sheets()]
    for sheet_name in sheet_names:    
        tmp_data = pd.read_excel(currentPath, sheet_name)
        tmp_data = tmp_data[tmp_data['R_W'] == 'Wrong']
        print('去空值前： ', tmp_data.shape, file_name, sheet_name)
        tmp_data = tmp_data[tmp_data['备注'] != '删除']
#         tmp_data = tmp_data.dropna(subset = ['raw_title'], axis = 0)
        print('去空值后： ', tmp_data.shape, file_name, sheet_name)
        cir_data_raw_2 = pd.concat([cir_data_raw_2, tmp_data], axis = 0)

cir_data_raw_2['label'] = cir_data_raw_2['label'].apply(lambda x:x.strip())  
cir_data_raw_2 = cir_data_raw_2[['备注', 'raw_title', 'raw_content']]
cir_data_raw_2.columns = ['label', 'title', 'content']
print('总数：', cir_data_raw_2.shape)
cir_data_raw_2.head()

去空值前：  (183, 7) 20180807_class_predict_result_check.xlsx 公司内部管理
去空值后：  (171, 7) 20180807_class_predict_result_check.xlsx 公司内部管理
去空值前：  (281, 7) 20180807_class_predict_result_check.xlsx 行业
去空值后：  (274, 7) 20180807_class_predict_result_check.xlsx 行业
去空值前：  (302, 7) 20180807_class_predict_result_check.xlsx 噪音
去空值后：  (298, 7) 20180807_class_predict_result_check.xlsx 噪音
去空值前：  (107, 7) 20180807_class_predict_result_check.xlsx 消费服务
去空值后：  (101, 7) 20180807_class_predict_result_check.xlsx 消费服务
去空值前：  (148, 7) 20180807_class_predict_result_check.xlsx 资本市场
去空值后：  (144, 7) 20180807_class_predict_result_check.xlsx 资本市场
去空值前：  (176, 7) 20180807_class_predict_result_check.xlsx 监管
去空值后：  (173, 7) 20180807_class_predict_result_check.xlsx 监管
去空值前：  (186, 7) 20180807_class_predict_result_check.xlsx 产品销售
去空值后：  (181, 7) 20180807_class_predict_result_check.xlsx 产品销售
去空值前：  (132, 7) 20180807_class_predict_result_check.xlsx 其他相关报道
去空值后：  (131, 7) 20180807_class_predict_result_check.xlsx 其他相关报道
总数： (1473, 3

,label,title,content
0,公司内部管理,最高可获赔付金100万元,南方日报讯 （记者/吴珂 通讯员/黄康灵 翟洪亮）1月5日，省公安厅与中国人民财产保险股份...
2,公司内部管理,中国人寿去年净利增68.6%,昨日晚间，中国人寿披露的2017年年报显示，公司去年实现净利润322.53亿元，同比...
3,其他相关报道,我国首家科技保险公司获批开业 注册地为浙江嘉兴,金融界保险1月9日讯 今日，中国保监会公布《关于太平科技保险股份有限公司开业的批复》\...
4,公司内部管理,腾讯微保被指违规经营 互联网保险业务边界屡受争议,腾讯微保被指违规经营 互联网保险业务边界屡受争议\n\r2018.01.10 09:04:...
5,公司内部管理,泰康资产管理_泰康资产管理公司_泰康资产怎么样,泰康人寿副总裁兼泰康资产首席投资官 邢怡\n\r中国网财经5月27日讯(记者 郭伟莹) 今...


In [13]:
cir_data_raw_2['label'].value_counts()

噪音        337
行业        300
公司内部管理    198
其他相关报道    169
监管        153
产品销售      137
资本市场      107
消费服务       72
Name: label, dtype: int64

## 新补充数据

In [14]:
file_list_3 = ['20180807_circ_class_1.xlsx', '20180808_circ_class_1.xlsx', 
               '20180808_circ_class_2.xlsx', '20180808_circ_class_3.xlsx',
               '20180809_circ_class_1.xlsx', '20180810_circ_class_1.xlsx'] # 多个 sheet

In [15]:
cir_data_raw_3 = pd.DataFrame()
for file_name in file_list_3:
    currentPath  = 'data/20180808/{0}'.format(file_name)
    excel = xlrd.open_workbook(currentPath)
    sheet_names = [sheet.name for sheet in excel.sheets()]
    for sheet_name in sheet_names:    
        tmp_data = pd.read_excel(currentPath, sheet_name)
        print(tmp_data.columns)
        if 'label' in tmp_data.columns:
            tmp_data = tmp_data[['label', 'title', 'content']]
            print('去空值前： ', tmp_data.shape, file_name, sheet_name)
            tmp_data = tmp_data.dropna(subset = ['title'], axis = 0)
            print('去空值后： ', tmp_data.shape, file_name, sheet_name)
            cir_data_raw_3 = pd.concat([cir_data_raw_3, tmp_data], axis = 0)

cir_data_raw_3['label'] = cir_data_raw_3['label'].apply(lambda x:x.strip())    
print('总数：', cir_data_raw_3.shape)
cir_data_raw_3.head()

Index(['title', 'content', 'label'], dtype='object')
去空值前：  (84, 3) 20180807_circ_class_1.xlsx 产品销售
去空值后：  (84, 3) 20180807_circ_class_1.xlsx 产品销售
Index(['title', 'content', 'label'], dtype='object')
去空值前：  (119, 3) 20180807_circ_class_1.xlsx 公司内部管理
去空值后：  (119, 3) 20180807_circ_class_1.xlsx 公司内部管理
Index(['title', 'content', 'label'], dtype='object')
去空值前：  (95, 3) 20180807_circ_class_1.xlsx 行业
去空值后：  (95, 3) 20180807_circ_class_1.xlsx 行业
Index(['title', 'content', 'label'], dtype='object')
去空值前：  (15, 3) 20180807_circ_class_1.xlsx 资本市场
去空值后：  (15, 3) 20180807_circ_class_1.xlsx 资本市场
Index(['id', 'predict_label', 'label', 'title', 'content', 'publishtime'], dtype='object')
去空值前：  (111, 3) 20180808_circ_class_1.xlsx Sheet1
去空值后：  (111, 3) 20180808_circ_class_1.xlsx Sheet1
Index(['title', 'content', 'label'], dtype='object')
去空值前：  (1038607, 3) 20180808_circ_class_1.xlsx Sheet4
去空值后：  (32, 3) 20180808_circ_class_1.xlsx Sheet4
Index(['label', 'title', 'content'], dtype='object')
去空值前：  (28

,label,title,content
0,产品销售,几款热销的性价比高消费型重疾险产品测评对比,几款热销的性价比高消费型重疾险产品测评对比 慧财365慧财365昨天 现...
1,产品销售,平安人寿推出“爱满分保险产品计划”,"专题平安人寿重磅推出新款少儿险“爱满分保险产品计划1”(以下简称“爱满分”),该产品针对0-..."
2,产品销售,少儿保障再升级 平安人寿“爱满分保险产品计划”上市,时六一儿童节之际，平安人寿重磅推出新款少儿险“爱满分保险产品计划1”（以下简称“爱满分”），...
3,产品销售,国寿少儿住院医疗保险，0免赔，不限社保，住院津贴补助，保费只需240元,投保须知 年龄 0岁-17岁 保期 1年 责任 疾病+意外+住院津贴多重保障，意外住院报销0...
4,产品销售,中国人寿推出国寿福（优享版）保险产品组合,"近日，寿险公司推出国寿福（优享版）保险产品组合,涵盖重大疾病、特定疾病、意外伤害、保费豁免等..."


In [16]:
cir_data_raw_3['label'].value_counts()

行业        356
资本市场      272
产品销售      252
公司内部管理    198
监管         49
消费服务       10
其他相关报道     10
Name: label, dtype: int64

## 模型预测后反馈数据

In [17]:
file_list_4 = ['20180810_circ_class_feedback.xlsx', 
              ] # 多个 sheet

In [18]:
cir_data_raw_4 = pd.DataFrame()
label_list = []
sum_list = []
right_list = []
for file_name in file_list_4:
    currentPath  = 'data/20180808/{0}'.format(file_name)
    excel = xlrd.open_workbook(currentPath)
    sheet_names = [sheet.name for sheet in excel.sheets()]
    for sheet_name in sheet_names:   
        tmp_data = pd.read_excel(currentPath, sheet_name)        
#         print(tmp_data.columns)
        tmp_data_1 = tmp_data[tmp_data['label'] == sheet_name]
        
        label_list.append(sheet_name)
        sum_list.append(tmp_data.shape[0])
        right_list.append(tmp_data_1.shape[0])
        print(sheet_name, tmp_data.shape[0], tmp_data_1.shape[0], tmp_data_1.shape[0]/tmp_data.shape[0])
        
        tmp_data_1 = tmp_data_1[['label', 'title', 'content']]
        print('去空值前： ', tmp_data_1.shape, file_name, sheet_name)
        tmp_data_1 = tmp_data_1.dropna(subset = ['title'], axis = 0)
        print('去空值后： ', tmp_data_1.shape, file_name, sheet_name)
        cir_data_raw_4 = pd.concat([cir_data_raw_4, tmp_data_1], axis = 0)
        print()

cir_data_raw_4['label'] = cir_data_raw_4['label'].apply(lambda x:x.strip())    
print('总数：', cir_data_raw_4.shape)
cir_data_raw_4.head()

行业 198 116 0.5858585858585859
去空值前：  (116, 3) 20180810_circ_class_feedback.xlsx 行业
去空值后：  (116, 3) 20180810_circ_class_feedback.xlsx 行业

监管 197 133 0.6751269035532995
去空值前：  (133, 3) 20180810_circ_class_feedback.xlsx 监管
去空值后：  (133, 3) 20180810_circ_class_feedback.xlsx 监管

其他相关报道 198 131 0.6616161616161617
去空值前：  (131, 3) 20180810_circ_class_feedback.xlsx 其他相关报道
去空值后：  (131, 3) 20180810_circ_class_feedback.xlsx 其他相关报道

噪音 198 117 0.5909090909090909
去空值前：  (117, 3) 20180810_circ_class_feedback.xlsx 噪音
去空值后：  (117, 3) 20180810_circ_class_feedback.xlsx 噪音

公司内部管理 194 117 0.6030927835051546
去空值前：  (117, 3) 20180810_circ_class_feedback.xlsx 公司内部管理
去空值后：  (117, 3) 20180810_circ_class_feedback.xlsx 公司内部管理

消费服务 195 140 0.717948717948718
去空值前：  (140, 3) 20180810_circ_class_feedback.xlsx 消费服务
去空值后：  (140, 3) 20180810_circ_class_feedback.xlsx 消费服务

资本市场 172 127 0.7383720930232558
去空值前：  (127, 3) 20180810_circ_class_feedback.xlsx 资本市场
去空值后：  (127, 3) 20180810_circ_class_feedback.xlsx 资本市场

产品销售 2

,label,title,content
0,行业,专栏文章|订单随意取消太任性？订单取消险了解一下！,前言 如今互联网的迅猛发展使得不少网购族享受到网上购买商品或者服务带来的便利。“七天无理由退...
1,行业,寿险公司二季度报告：前海人寿等四家险企亏损严重,一半是海水，一半是火焰。\n\n 根据21世纪经济报道记者统计，截至8月5日，在72家统...
2,行业,【贝叶思报告】2018年保险业声誉风险管理周报（7月21日-7月27日）,一、本周舆情概述 2018年7月21日-7月27日，贝叶思监测数据显示，全网有关保险行业的声...
3,行业,“信保贷”在保项目金额为54.3亿元,有效降低贷款业务风险。保险机构有3家。 除了“信保贷”项目，我市上半年其他保险重点项目...
4,行业,健康险发展缓慢--,今日，中信保诚人寿在客服节启动仪式上首发《国民健康大数据白皮书》，其中显示，虽然疾病费用开销...


In [19]:
cir_data_raw_4['label'].value_counts()

消费服务      140
监管        133
其他相关报道    131
资本市场      127
产品销售      122
公司内部管理    117
噪音        117
行业        116
Name: label, dtype: int64

## 合并数据

In [20]:
cir_data_raw = pd.concat([cir_data_raw_1, cir_data_raw_2,
                          cir_data_raw_3, cir_data_raw_4], 
                         axis = 0)

# cir_data_raw = cir_data_raw_2
cir_data_raw['title_content'] = cir_data_raw['title'].astype(str) + '。' + cir_data_raw['content'].astype(str)
cir_data_raw['label'] = cir_data_raw['label'].apply(lambda x:x.strip())
cir_data_raw['index'] = range(cir_data_raw.shape[0])
print('去重前： ', cir_data_raw.shape)
cir_data_raw = cir_data_raw.drop_duplicates(subset = 'title_content')
print('去重后： ', cir_data_raw.shape)
cir_data_raw.head()

去重前：  (12519, 5)
去重后：  (11979, 5)


,label,title,content,title_content,index
2,公司内部管理,中华保险因未按规定报送可疑交易报告等遭央行罚9万,中华保险因未按规定报送可疑交易报告等遭央行罚9万\n\r2018.01.04 09:17:...,中华保险因未按规定报送可疑交易报告等遭央行罚9万。 中华保险因未按规定报送可疑交易报告等遭央...,0
4,公司内部管理,中国人寿北京违法欺骗投保人_保险_金融频道首页_财经网,中国经济网北京1月2日讯北京保监局于2017年12月27日对中国人寿保险股份有限公司北京市...,中国人寿北京违法欺骗投保人_保险_金融频道首页_财经网。 中国经济网北京1月2日讯北京保监局...,1
5,公司内部管理,中国人寿北京市分公司因欺骗投保人被保监局罚款30万元,中国人寿\n\r北京市分公司因欺骗投保人被保监局罚款30万元\n\r千龙网北京1月2日讯 ...,中国人寿北京市分公司因欺骗投保人被保监局罚款30万元。 中国人寿\n\r北京市分公司因欺骗投...,2
6,公司内部管理,中国人寿|北京保监局_新浪财经_新浪网,中国经济网北京1月2日讯 北京保监局于2017年12月27日对\n\r中国人寿\n\r...,中国人寿|北京保监局_新浪财经_新浪网。 中国经济网北京1月2日讯 北京保监局于2017...,3
8,公司内部管理,中国人保分公司涉嫌编制虚假报告 被保监会罚款27万,中国人保分公司涉嫌编制虚假报告 被保监会罚款27万\n\r1\n评论\n\r2018-01...,中国人保分公司涉嫌编制虚假报告 被保监会罚款27万。 中国人保分公司涉嫌编制虚假报告 被保监...,4


In [21]:
print(cir_data_raw.shape)
cir_data_raw.groupby('label')['title'].count()

(11979, 5)


label
产品销售       669
公司内部管理    1161
其他相关报道    1571
噪音        3474
消费服务      1521
监管        1519
行业        1201
资本市场       863
Name: title, dtype: int64

In [22]:
test_index = []
val_index = []
train_index = []

for label in cir_data_raw['label'].unique():
    indexs = cir_data_raw[cir_data_raw['label'] == label]['index'].tolist()
    test_index += indexs[:40] # 40
    val_index += indexs[40:60] # 20
    train_index += indexs[60:]

In [23]:
coprus_save_filename = 'data/cnews.test.txt'

test_data = np.array(cir_data_raw[cir_data_raw['index'].isin(test_index)][['label', 'title_content']])
print(test_data.shape)

f = open(coprus_save_filename, "w+", encoding='UTF-8')
for [l, d] in test_data:
    f.write(l + '----' + d + '\n')
f.close()

(320, 2)


In [24]:
coprus_save_filename = 'data/cnews.val.txt'

val_data = np.array(cir_data_raw[cir_data_raw['index'].isin(val_index)][['label', 'title_content']])
print(val_data.shape)

f = open(coprus_save_filename, "w+", encoding='UTF-8')
for [l, d] in val_data:
    f.write(l + '----' + d + '\n')
f.close()

(160, 2)


In [25]:
coprus_save_filename = 'data/cnews.train.txt'

train_data = np.array(cir_data_raw[cir_data_raw['index'].isin(train_index)][['label', 'title_content']])
print(train_data.shape)

f = open(coprus_save_filename, "w+", encoding='UTF-8')
for [l, d] in train_data:
    f.write(l + '----' + d + '\n')
f.close()

(11499, 2)


In [26]:
label_dic={'监管':1,'行业':2,'产品销售':3,'资本市场':4,'公司内部管理':5,'消费服务':6,'其他相关报道':7,'噪音':8}
label_dic.keys()

dict_keys(['监管', '行业', '产品销售', '资本市场', '公司内部管理', '消费服务', '其他相关报道', '噪音'])

# 预处理

In [64]:
# titles = pre_cor_1.handle_contents(cir_data_raw['title'].tolist())
titles = pre_cor.handle_contents(cir_data_raw['title'].tolist())
print(len(titles))

save_filename = 'data/titles0.txt'
fid = open(save_filename, "w+", encoding='UTF-8')
for data in titles:
    fid.write(data + '\n')
fid.close()

11979


In [65]:
# contents = pre_cor_1.handle_contents(cir_data_raw['content'].tolist())
contents = pre_cor.handle_contents(cir_data_raw['content'].tolist())
print(len(contents))
print(contents[:2])
contents = [re.sub(r'[a-z]*', '', x) for x in contents]
print(len(contents))
print(contents[:2])

coprus_save_filename = 'data/contents0.txt'
f = open(coprus_save_filename, "w+", encoding='UTF-8')
for data in contents:
    f.write(data + '\n')
f.close()

11979
['中华保险 因未 按规定 报送 可疑交易 报告 遭 央行 罚 万 n r n 新浪 综合 n r 图片 n r 千 新闻网 n r 月 日 中国人民银行 阿坝州 中心 支行 发布 行政处罚 信息 公示 表 显示 中华联合财产保险股份有限公司 阿坝 中心 支公司 因 存在 未 按照 规定 履行 客户 身份 识别 义务 未 按照 规定 报送 可疑交易 报告 行为 被 处以罚款 万元 相关 责任人 被 处以罚款 万元 作出 行政处罚 决定 日期 为 年 月 日 n r 图片 r n t t t t t t t t t 图片 n r 资料 显示 中华联合财产保险股份有限公司 最早 成立 年 前身 为 国家财政部 农业部 拔 专款 新疆生产建设兵团 组建 成立 新疆 兵团 保险公司 年 更名 为 中华联合财险 年 实行 一改 三 整体 改制 成立 中华 联合 控股公司 下设 中华联合财险 中华联合人寿 n r 责任编辑 杜琰 SF n r 图片 n ', '中国 经济网 北京 月 日讯 北京保监局 年 月 日 对 中国人寿保险股份有限公司 北京市 分公司 以下 简称 国寿北分 作出 京 保监 罚 号 处罚 决定 并 今日 公布 北京保监局 核查 国寿北分 经营 过程 中 存在 欺骗 投保人 违法行为 n r 北京保监局 表示 国寿北分 欺骗 投保人 行为 违反 中华人民共和国 保险法 一百一十 六条 规定 依据 该法 第一百六十一条 对国寿北分 处以罚款 万元 行政处罚 并 责令 改正 违法违规 行为 n r 此外 时任 国寿北分 副总经理 张 松山 身份证号 XXXXXXXX 时任 国寿北分 银行 保险 部 总经理 程炳华 身份证号 XXXXXXXX 对 上述 违规 问题 直接 负责 主管人员 时任 国寿北分 银行 保险 部 理财 管理部 经理 朱宇阳 身份证号 XXXXXXXX 上述 违规 问题 直接 责任人员 北京保监局 根据 中华人民共和国 保险法 一百七十 一条 决定 对张 松山 给予警告 并处 万元 罚款 行政处罚 对 程炳华 给予警告 并处 万元 罚款 行政处罚 对 朱宇阳 给予警告 并处 万元 罚款 行政处罚 n r 图片 n r 编辑 许楠楠 n r 关键字 n r 投保人 n r 北京 n r 人寿 n r 中国 n r 违法 n ']
11

In [66]:
label = cir_data_raw['label'].tolist()
print(len(label))

coprus_save_filename = 'data/labels.txt'
f = open(coprus_save_filename, "w+", encoding='UTF-8')
for data in label:
    f.write(data + '\n')
f.close()

11979


In [67]:
cir_data_raw.to_excel('data/result/title_content_label.xlsx', index = False)
cir_data_raw.shape

(11979, 5)